In [8]:
import pyranges as pr
import pandas as pd
import numpy as np
import h5py 
import os

from cerberus.cerberus import *
from cerberus.main import *


## convert transcriptome

In [9]:
def add_triplets(t_df, ic, tss, tes):
    """
    Determines which tss, intron chain, and tes are used from a cerberus
    reference are used in a specific gtf.

    Parameters:
        gtf (str): File path to gtf you want to assign triplets to
        ic_file (str): File path to intron chain tsv (part of cerberus ref.)
        tss_bed (str): File path to tss bed (part of cerberus ref.)
        tes_bed (str): File path to tes bed (part of cerberus ref.)

    Returns:
        df (pandas DataFrame): File that maps each transcript from gtf to
            a tss, intron chain, and tes given by the cerberus reference.
            Also includes new transcript ids to refer to each transcript
    """

    ### intron chain annotation ###

    # get unique intron chains from gtf
    df = t_df.copy()
    df = get_ic(df)
    df.rename({'ic': 'Coordinates'},
              axis=1, inplace=True)

    # read in reference set of intron chains
    ref = ic

    # merge on intron chain, strand, chromosome, and gene id
    df = df.merge(ref, how='left',
                 on=['Chromosome', 'Strand',
                     'Coordinates', 'gene_id'])

    # formatting
    df.rename({'Name': 'ic_id'}, axis=1, inplace=True)
    df = df[['transcript_id', 'ic', 'ic_id']]

    ### end annotation ###

    for mode, ref in zip(['tss', 'tes'], [tss, tes]):

        if mode == 'tss':
            ends = t_df.features.tss()
        elif mode == 'tes':
            ends = t_df.features.tes()

        # merge transcriptome ends with reference ends
        ref = read_cerberus_ends(bed_file, mode=mode)
        ref = pr.PyRanges(ref)
        ends = ends.join(ref,
                         strandedness='same',
                         how='left').df
        ends = ends.loc[ends.gene_id == ends.gene_id_b]

        # formatting
        ends.rename({'Name': '{}_id'.format(mode)}, axis=1, inplace=True)
        ends = ends[['transcript_id', mode, '{}_id'.format(mode)]]

        # add ends into map df
        df = df.merge(ends, how='left', on='transcript_id')

    ### creating map file ###

    # get gene id / name and transcript name from original gtf
    t_df = t_df.df
    t_df = t_df.loc[t_df.Feature == 'transcript']
    if 'gene_name' not in t_df.columns:
        t_df['gene_name'] = t_df.transcript_name.str.split('-', n=1, expand=True)[0]
    t_df = t_df[['gene_id', 'gene_name',
                 'transcript_id', 'transcript_name']]
    df = df.merge(t_df, how='left', on='transcript_id')

    # create triplets and rename old ids
    df.rename({'transcript_id': 'original_transcript_id',
               'transcript_name': 'original_transcript_name'},
               axis=1, inplace=True)
    df['transcript_triplet'] = '['+df.tss.astype(str)+', '+\
                                   df.ic.astype(str)+', '+\
                                   df.tes.astype(str)+']'
    df['transcript_id'] = df['gene_id']+' '+df.transcript_triplet
    df['transcript_name'] = df['gene_name']+' '+df.transcript_triplet

    return df

In [10]:
def convert_transcriptome_command(gtf, h5):
    convert_transcriptome(gtf, h5)
    
def convert_transcriptome(gtf, h5):
    
    # read in / format existing reference
    ic, tss, tes, _ = read_h5(h5, as_pyranges=False)
    ic = split_cerberus_id(ic, 'ic')
    tss = split_cerberus_id(tss, 'tss')
    tes = split_cerberus_id(tes, 'tes')
    tss = pr.PyRanges(tss)
    tes = pr.PyRanges(tes)
    
    # read in transcriptome to convert to cerberus
    gtf_df = pr.read_gtf(gtf)
    
    return gtf_df, tss, ic, tes
    
#     # assign triplets to each transcript from the gtf from the reference
#     df = assign_triplets(t_df, ic, tss, tes)
    
#     print()
#     print(ic.head())
#     print(tss.head())
#     print(tes.head())
#     print()
#     print(t_df.head())
    

    
    

In [11]:
# def assign_triplets(gtf_df, tss, ic, tes):
    
#     # get intron chains from gtf
#     gtf_df = 

In [17]:
h5 = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/cerberus_ref.h5'
gtf = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/talon/human_known_nic_nnc_talon.gtf'
# gtf_df, tss, ic, tes = convert_transcriptome_command(gtf, h5)
gtf_df, tss, ic, tes = convert_transcriptome(gtf, h5)



In [18]:
gtf_df = gtf_df.df
gtf_df['gene_id'] = gtf_df.gene_id.str.split('.', n=1, expand=True)[0]
gtf_df = pr.PyRanges(gtf_df)

In [19]:
m_df = get_ic(gtf_df)
# print(m_df.head())
m_df.rename({'ic': 'Coordinates'}, axis=1, inplace=True)
m_df = m_df.merge(ic, how='left',
                  on=['Chromosome', 'Strand', 
                      'Coordinates', 'gene_id'])

# # formatting
# m_df.rename({'Name': 'ic_id'}, axis=1, inplace=True)
# m_df = m_df[['transcript_id', 'ic_id']]

In [20]:
m_df.head()

,Chromosome,Strand,transcript_id,gene_id,Coordinates,Name,source,ic
0,SIRV1,+,ENCODEHT000206942,ENCODEHG000058846,10791-10882-11057-11434,ENCODEHG000058846_1,talon,1
1,SIRV1,-,ENCODEHT000206867,ENCODEHG000058837,10647-10366-10282-7814-7552-6813-6560-6473-633...,ENCODEHG000058837_4,talon,4
2,SIRV1,-,ENCODEHT000206868,ENCODEHG000058837,10444-10366-10282-7814-7552-6813-6560-6473-633...,ENCODEHG000058837_2,talon,2
3,SIRV1,-,ENCODEHT000206870,ENCODEHG000058837,10553-7808-7552-1484,ENCODEHG000058837_3,talon,3
4,SIRV1,-,ENCODEHT000206886,ENCODEHG000058837,10282-7814-7552-6813-6337-1484,ENCODEHG000058837_1,talon,1


In [2]:
gtf_to_ics('/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/talon/human_known_nic_nnc_talon.gtf', o='test.tsv')

  Chromosome Strand              Coordinates            gene_id  \
0      SIRV1      +  10791-10882-11057-11434  ENCODEHG000058846   

       transcript_id  MANE_Select  basic_set  appris_principal  ic_num  \
0  ENCODEHT000206942        False      False               NaN       1   

                  Name  
0  ENCODEHG000058846_1  


In [21]:
h5 = '/Users/fairliereese/mortazavi_lab/data/rnawg/lr_bulk/cerberus/cerberus_ref.h5'
ic, tss, tes, _ = read_h5(h5)
write_h5(ic, tss, tes, h5)
ic, tss, tes, _ = read_h5(h5)

In [25]:
print(ic.loc[ic.Coordinates=='10791-10882-11057-11434'])
print(len(ic.loc[ic.Name.str.contains('ENSG')]))
print(len(ic.loc[ic.Name.str.contains('ENCODE')]))
print(len(ic.loc[~(ic.Name.str.contains('ENSG'))&~(ic.Name.str.contains('ENCODE'))]))
ic.loc[~(ic.Name.str.contains('ENSG'))&~(ic.Name.str.contains('ENCODE'))].head()
# uhhhh I don't think that there are any sirv /ercc / long sirv genes in our talon stuff oops


       Chromosome Strand              Coordinates                 Name source
241882      SIRV1      +  10791-10882-11057-11434             SIRV1B_1    v29
0           SIRV1      +  10791-10882-11057-11434  ENCODEHG000058846_1  talon
283060
441
175


,Chromosome,Strand,Coordinates,Name,source
241790,ERCC-00002,+,1045,ERCC-00002A_1,v29
241791,ERCC-00003,+,1007,ERCC-00003A_1,v29
241792,ERCC-00004,+,507,ERCC-00004A_1,v29
241793,ERCC-00009,+,968,ERCC-00009A_1,v29
241794,ERCC-00012,+,978,ERCC-00012A_1,v29


In [12]:
# ?write_h5

Signature: write_h5(ic, tss, tes, oname, m=None)
Docstring:
Write a cerberus transcriptome as an h5df file

Parameters:
    ic (pandas DataFrame): DataFrame of intron chains
    tss (pandas DataFrame): DataFrame in bed format of tsss
    tes (pandas DataFrame): DataFrame in bed format of tess
    oname (str): Output file name ending in '.h5'
    m (pandas DataFrame): DataFrame of map file
File:      ~/Documents/programming/mortazavi_lab/bin/cerberus/cerberus/cerberus.py
Type:      function
